In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

In [2]:
ratings = np.zeros((200, 1000))
ratings_tf = np.full((200, 1000), False)
train = open("train.txt", "r")
for t in train:
    user, movie, rating = t[:-1].split(" ")
    user, movie, rating = int(user), int(movie), int(rating)
    ratings[user-1][movie-1] = rating
    ratings_tf[user-1][movie-1] = True
train.close()
user_averages = [sum(ratings[i])/sum(ratings_tf[i]) for i in range(len(ratings))]

In [3]:
test5_ratings = np.zeros((100, 1000))
test5_ratings_tf = np.full((100, 1000), False)
test5 = open("test5.txt", "r")
for t in test5:
    user, movie, rating = t[:-1].split(" ")
    user, movie, rating = int(user), int(movie), int(rating)
    test5_ratings[user-201][movie-1] = rating
    test5_ratings_tf[user-201][movie-1] = True
test5.close()

In [4]:
test10_ratings = np.zeros((100, 1000))
test10_ratings_tf = np.full((100, 1000), False)
test10 = open("test10.txt", "r")
for t in test10:
    user, movie, rating = t[:-1].split(" ")
    user, movie, rating = int(user), int(movie), int(rating)
    test10_ratings[user-301][movie-1] = rating
    test10_ratings_tf[user-301][movie-1] = True
test10.close()

In [5]:
test20_ratings = np.zeros((100, 1000))
test20_ratings_tf = np.full((100, 1000), False)
test20 = open("test20.txt", "r")
for t in test20:
    user, movie, rating = t[:-1].split(" ")
    user, movie, rating = int(user), int(movie), int(rating)
    test20_ratings[user-401][movie-1] = rating
    test20_ratings_tf[user-401][movie-1] = True
test20.close()

In [6]:
test_ratings_list = [test5_ratings, test10_ratings, test20_ratings]
test_ratings_tf_list = [test5_ratings_tf, test10_ratings_tf, test20_ratings_tf]

In [7]:
result5_user_cosine = open("result5_user_cosine.txt", "w")
result10_user_cosine = open("result10_user_cosine.txt", "w")
result20_user_cosine = open("result20_user_cosine.txt", "w")

result5_user_pearson = open("result5_user_pearson.txt", "w")
result10_user_pearson = open("result10_user_pearson.txt", "w")
result20_user_pearson = open("result20_user_pearson.txt", "w")

result5_item_cosine = open("result5_item_cosine.txt", "w")
result10_item_cosine = open("result10_item_cosine.txt", "w")
result20_item_cosine = open("result20_item_cosine.txt", "w")

result_files_user_cosine = [result5_user_cosine, result10_user_cosine, result20_user_cosine]
result_files_user_pearson = [result5_user_pearson, result10_user_pearson, result20_user_pearson]
result_files_item_cosine = [result5_item_cosine, result10_item_cosine, result20_item_cosine]

In [8]:
def cosine_similarity(x, y):
    numerator = sum(x*y)
    denominator = math.sqrt(sum(x**2)*sum(y**2))
    cosine_sim = numerator/denominator
    return cosine_sim

In [9]:
def adjusted_cosine_similarity(x, y, l):
    x = x - l
    y = y - l
    numerator = sum(x*y)
    denominator = math.sqrt(sum(x**2)*sum(y**2))
    adjusted_cosine_sim = numerator/denominator
    return adjusted_cosine_sim

In [10]:
def pearson_correlation(x, y):
    x_avg = sum(x)/len(x)
    y_avg = sum(y)/len(y)
    x = x-x_avg
    y = y-y_avg
    numerator = sum(x*y)
    denominator = math.sqrt(sum(x**2)*sum(y**2))
    pearson_corr = numerator/denominator
    return pearson_corr

# User-Based Collaborative Filtering Algorithms

In [11]:
user_similarity_array_cosine = np.zeros((ratings.shape[0])) #user similarity matrix
user_similarity_array_pearson = np.zeros((ratings.shape[0]))
for t in range(len(test_ratings_list)):
    file_cosine = result_files_user_cosine[t]
    file_pearson = result_files_user_pearson[t]
    n = -1
    for i in range(len(test_ratings_list[t])):
        current_user_avg_rating = sum(test_ratings_list[t][i])/sum(test_ratings_tf_list[t][i])
        for j in range(ratings.shape[0]):
            user_similarity_array_cosine[j] = cosine_similarity(ratings[j], test_ratings_list[t][i])
            user_similarity_array_pearson[j] = pearson_correlation(ratings[j], test_ratings_list[t][i])
        s1 = 30#number of similar users(cosine similarity)
        s2 = 30#number of similar users(pearson correlation)
        most_similar_users_cosine = np.argsort(user_similarity_array_cosine)[-s1:]
        most_similar_users_pearson = np.argsort(user_similarity_array_pearson)[-s2:]
        for j in range(ratings.shape[1]):
            if test_ratings_list[t][i][j] == 0 and test_ratings_tf_list[t][i][j]:
                n += 1
                numerator = 0
                denominator = 0
                numerator_pearson = 0
                denominator_pearson = 0
                for similar_user in most_similar_users_cosine:
                    if ratings[similar_user][j] != 0:
                        numerator += user_similarity_array_cosine[similar_user]*ratings[similar_user][j]#ratings or ratings_new?
                        denominator += user_similarity_array_cosine[similar_user]
                        numerator_pearson += abs(user_similarity_array_pearson[similar_user])*abs(ratings[similar_user][j] - user_averages[similar_user])
                        denominator_pearson += abs(user_similarity_array_pearson[similar_user])
                pred_rating_cosine = numerator/denominator if denominator != 0 else random.randint(1, 5) #else random int between 1 and 5
                if denominator_pearson != 0:
                    pred_rating_pearson = current_user_avg_rating + (numerator_pearson/denominator_pearson)
                else:
                    pred_rating_pearson = random.randint(1, 5)
                a, b, c = str(int(i + 100*t + 201)), str(int(j + 1)), str(int(round(pred_rating_cosine)))
                d = int(round(abs(pred_rating_pearson)))
                if d == 0:
                    d = str("1")
                else:
                    d = str(d)
                line_cosine = a + " " + b + " " + c
                line_pearson = a + " " + b + " " + d
                file_cosine.write(line_cosine+"\n")
                file_pearson.write(line_pearson+"\n")
    file_cosine.close()
    file_pearson.close()

# Item-based collaborative filtering

In [12]:
for t in range(len(test_ratings_list)):
    file_cosine = result_files_item_cosine[t]
    for i in range(len(test_ratings_list[t])):
        ratings_new = np.concatenate((ratings, [test_ratings_list[t][i]]), axis=0)#concatenate
        ratings_transpose = np.transpose(ratings_new)#transpose
        curr_user_avg_rating = sum(ratings_new[200])/sum(test_ratings_tf_list[t][i])
        l = user_averages[:]
        l.append(curr_user_avg_rating)
        for j in range(ratings_transpose.shape[0]):
            if ratings_transpose[j][200] == 0 and test_ratings_tf_list[t][i][j]:
                item_similarity_array_cosine = np.zeros(ratings_transpose.shape[0])
                for k in range(ratings_transpose.shape[0]):
                    item_similarity_array_cosine[k] = adjusted_cosine_similarity(ratings_transpose[k], ratings_transpose[j], l)
                
                most_similar_items = np.argsort(item_similarity_array_cosine)[-20:]
                numerator = 0
                denominator = 0
                for similar_item in most_similar_items:
                    if ratings_transpose[similar_item][-1] != 0:
                        numerator += item_similarity_array_cosine[similar_item]*ratings_transpose[similar_item][-1]
                        denominator += item_similarity_array_cosine[similar_item]
                if denominator != 0:
                    if numerator == 0:
                        pred_rating = user_averages[i]
                    else:
                        pred_rating = numerator/denominator
                else:
                    pred_rating = user_averages[i]
                a, b, c = str(int(i + 100*t + 201)), str(int(j + 1)), str(int(round(pred_rating)))
                cosine_line = a + " " + b + " " + c
                file_cosine.write(cosine_line+"\n")
    file_cosine.close()